In [50]:
import django_initializer  # noqa: F401
from telemetry.models import Lap
from telemetry.influx import Influx
import plotly.express as px
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots
import plotly.graph_objects as go
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from telemetry.visualizer import *
from telemetry.analyzer import Analyzer

influx = Influx()
analyzer = Analyzer()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2024-01-14 13:41:52,495 DEBUG Influx: Connected to https://telemetry.b4mad.racing:443/


In [51]:
measurement = "fast_laps"
bucket = "fast_laps"
# measurement = "laps_cc"
# bucket = "racing"
lap = Lap.objects.get(id=2084)
session_df = influx.telemetry_for_laps([lap], measurement=measurement, bucket=bucket)[0]

2024-01-14 13:41:54,812 INFO Fetching telemetry for iRacing - spielberg gp - Mercedes-AMG GT3 2020
2024-01-14 13:41:54,813 INFO   track.id 16 car.id 28
2024-01-14 13:41:54,813 INFO   session 1680444939 lap.id 2084 number 2
2024-01-14 13:41:54,813 INFO   length 4279 time 89.8657 valid True
2024-01-14 13:41:54,814 INFO   start 2023-04-02 16:16:25.562000+00:00 end 2023-04-02 16:17:55.443000+00:00


In [64]:
df = session_df.copy()

resampled_df = analyzer.resample_channels(df, ["Throttle", "Gear"], 1)
fig = lap_fig(resampled_df, full_range=True, columns=['Throttle'])
fig.show()
resampled_df


,DistanceRoundTrack,Throttle,Gear
0,0.0,1.0,4.0
1,1.0,1.0,4.0
2,2.0,1.0,4.0
3,3.0,1.0,4.0
4,4.0,1.0,4.0
...,...,...,...
4275,4275.0,1.0,4.0
4276,4276.0,1.0,4.0
4277,4277.0,1.0,4.0
4278,4278.0,1.0,4.0


In [47]:
df = session_df.copy()

# display(df.info())

columns = ['DistanceRoundTrack', 'Throttle', 'Brake']
linear_df = df[columns]

columns = ['DistanceRoundTrack', 'Gear']
discrete_df = df[columns]

# Define the new distance range
new_distance_interval = 1  # meters
max_distance = lap.track.length
print(f"Max distance: {max_distance}")
print(f"Session min distance: {session_df['DistanceRoundTrack'].min()}")
print(f"Session max distance: {session_df['DistanceRoundTrack'].max()}")

new_distance = np.arange(0, max_distance + new_distance_interval, new_distance_interval)
# Ensure new_distance is of type float
new_distance = new_distance.astype(float)

# Create a new DataFrame with evenly spaced Distance
new_df = pd.DataFrame(new_distance, columns=['DistanceRoundTrack'])

# The pd.merge_asof() function is used for merging on near keys. The interpolate() function then fills in the missing 'Throttle' values. The interpolation method can be adjusted based on your data characteristics (e.g., linear, quadratic).
# Merge and interpolate
resampled_df = pd.merge_asof(new_df, linear_df, on='DistanceRoundTrack', direction='nearest').interpolate()
resampled_df = pd.merge_asof(resampled_df, discrete_df, on='DistanceRoundTrack', direction='nearest').bfill()

display(resampled_df['Gear'].unique())

display(resampled_df)
fig = lap_fig(resampled_df, full_range=True, columns=['Throttle'])
fig.show()
fig2 = lap_fig(df, full_range=True, columns=['Throttle'])
fig2.show()

# display(df)
# display(resampled_df)


Max distance: 4280
Session min distance: 0.37535435
Session max distance: 4279.46729


array([4., 5., 3., 2., 1.])

,DistanceRoundTrack,Throttle,Brake,Gear
0,0.0,1.0,0.0,4.0
1,1.0,1.0,0.0,4.0
2,2.0,1.0,0.0,4.0
3,3.0,1.0,0.0,4.0
4,4.0,1.0,0.0,4.0
...,...,...,...,...
4276,4276.0,1.0,0.0,4.0
4277,4277.0,1.0,0.0,4.0
4278,4278.0,1.0,0.0,4.0
4279,4279.0,1.0,0.0,4.0


In [6]:
# Example DataFrame
data = {
    'time': pd.date_range(start='2024-01-01', periods=5, freq='T'),
    'DistanceRoundTrack': [20, 500, 1200, 1800, 2500],  # in meters
    'Throttle': [0.2, 0.6, 0.8, 0.7, 0.5]
}
df = pd.DataFrame(data)

# Define the new distance range
new_distance_interval = 1  # meters
max_distance = df['DistanceRoundTrack'].max()
new_distance = np.arange(0, max_distance + new_distance_interval, new_distance_interval)

# Create a new DataFrame with evenly spaced Distance
new_df = pd.DataFrame(new_distance, columns=['DistanceRoundTrack'])

# Merge and interpolate
resampled_df = pd.merge_asof(new_df, df, on='DistanceRoundTrack', direction='nearest').interpolate()

print(resampled_df)


      DistanceRoundTrack                time  Throttle
0                      0 2024-01-01 00:00:00       0.2
1                      1 2024-01-01 00:00:00       0.2
2                      2 2024-01-01 00:00:00       0.2
3                      3 2024-01-01 00:00:00       0.2
4                      4 2024-01-01 00:00:00       0.2
...                  ...                 ...       ...
2496                2496 2024-01-01 00:04:00       0.5
2497                2497 2024-01-01 00:04:00       0.5
2498                2498 2024-01-01 00:04:00       0.5
2499                2499 2024-01-01 00:04:00       0.5
2500                2500 2024-01-01 00:04:00       0.5

[2501 rows x 3 columns]
